### Load the MI data

In [302]:
import numpy as np
seed = 246
a = np.load(f'mi_all_4_{seed}.npy')
#a = np.load("results/final_tanh_246_sd01/mi_all_4_246.npy")
print(a.shape)


(2, 109, 4)


### Print a part of the data

In [303]:
mi_xt_mean = a[0]
mi_ty_mean = a[1]
#mi_xt_mean = a[0,0,:,:]
#mi_ty_mean = a[0,1,:,:]
print(mi_xt_mean.shape)
print(mi_xt_mean)
print(mi_ty_mean)

(109, 4)
[[13.21156853  5.53771277  3.69010577  1.32395026]
 [13.02027069  5.65834157  4.11665885  2.19623657]
 [12.87057353  5.54944687  4.10321307  2.76784165]
 [12.65370376  5.43852378  3.76935288  2.80472724]
 [12.61679908  5.45733473  3.50542805  2.90135878]
 [12.5027399   5.20451626  3.0948843   2.66925213]
 [12.23626677  5.15264212  2.99810406  2.9154047 ]
 [12.44228686  5.14797115  2.89462159  2.93667805]
 [12.50819998  4.99221929  2.6978728   2.78457117]
 [12.35818285  4.7573552   2.60499482  2.75176828]
 [12.34261883  4.72810415  2.53088942  2.66313613]
 [12.43453313  4.46864779  2.30749806  2.58268723]
 [12.2353039   4.38939038  2.32139983  2.78796106]
 [12.35081565  4.36632429  2.41201673  2.95714446]
 [12.28313045  4.13095297  2.10890647  2.96487378]
 [12.03142983  4.20672832  2.08408542  2.9131392 ]
 [12.06738961  4.10686259  2.02658522  2.9777392 ]
 [11.9343628   3.90174059  2.02824779  3.21402839]
 [11.92967467  3.81524898  1.87494735  2.83405963]
 [12.25065548  3.88623

### Sample from the MI data

In [304]:
IXM_N = mi_xt_mean[0,:]
IYM_N = mi_ty_mean[0,:]

T =1

for i in range(1,mi_xt_mean.shape[0]):
    
    A_ = i <= 4 and i % 1 == 0  
    A0 = i > 4 and i <= 10 and i % 2 == 0    
    A1 = i > 10 and i <= 30 and i % 3 == 0     
    A2 = i > 30 and i % 3 == 0    

    #if A0 or A1 or A2:
    if  A0 or A1 or A2:
        mean_vec_xt = np.mean(mi_xt_mean[T:i+1,:],axis=0)
        mean_vec_yt = np.mean(mi_ty_mean[T:i+1,:],axis=0)  
                             
        #print(T,i+1)                      
        IXM_N = np.append(IXM_N,mean_vec_xt)
        IYM_N = np.append(IYM_N,mean_vec_yt)
        
        T = i+1

        
IXM_N = np.reshape(IXM_N,(-1,4))
IYM_N = np.reshape(IYM_N,(-1,4))
print(IXM_N.shape)

(37, 4)


### Data shape

In [305]:
#mi_ty_mean[0:10,3]=np.array(mi_ty_mean[0:10,3])
IXM_N, IYM_N = mi_ty_mean, mi_ty_mean
print(IXM_N.shape)
print(IYM_N.shape)

(109, 4)
(109, 4)


### Apply roling mean

In [306]:
import pandas as pd
T=3
IXM_N, IYM_N = np.zeros((mi_xt_mean.shape[0]-T+1,4)), np.zeros((mi_ty_mean.shape[0]-T+1,4))

for i in range(4):
    IXM_N[:, i] = pd.Series(mi_xt_mean[:, i]).rolling(window=T).mean()[T-1:].to_numpy()
    IYM_N[:, i] = pd.Series(mi_ty_mean[:, i]).rolling(window=T).mean()[T-1:].to_numpy()


    
print(IXM_N.shape)
print(IYM_N.shape)

(107, 4)
(107, 4)


### Plot Information Plane

In [186]:


#%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
sns.set_style('darkgrid')


plt.figure(figsize=(8,8))
gs = gridspec.GridSpec(4,2)
#for actndx, (activation, vals) in enumerate(measures.items()):
#    epochs = sorted(vals.keys())
#    if not len(epochs):
#        continue

COLORBAR_MAX_EPOCHS=5000 # initially 15000

plt.tight_layout()        

fig=plt.figure(figsize=(10,5))

#max_epoch = max( (max(vals.keys()) if len(vals) else 0) for vals in measures.values())
sm = plt.cm.ScalarMappable(cmap='gnuplot', norm=plt.Normalize(vmin=0, vmax=COLORBAR_MAX_EPOCHS))
sm._A = []

#fig=plt.figure(figsize=(10,5))
#for actndx, (activation, vals) in enumerate(measures.items()):
#    epochs = sorted(vals.keys())
#    if not len(epochs):
#        continue
#    plt.subplot(1,2,actndx+1)    
    
    
#n_epoch = mi_xt_mean.shape[0]
n_epoch = IXM_N.shape[0]

PLOT_LAYERS = [0]
for epoch in range(0,76): # from n_epoch, 76 for first 1000
    c = sm.to_rgba(220*(epoch))
    #xmvals = np.array(vals[epoch]['MI_XM'])[PLOT_LAYERS]
    #ymvals = np.array(vals[epoch]['MI_YM'])[PLOT_LAYERS]

    #xmvals = mi_xt_mean[epoch] 
    #ymvals = mi_ty_mean[epoch]
    xmvals = IXM_N[epoch] 
    ymvals = IYM_N[epoch]
    
    plt.plot(xmvals, ymvals, c=c, alpha=0.1, zorder=1)
    plt.scatter(xmvals, ymvals, s=40, facecolors=[c for _ in PLOT_LAYERS], edgecolor='none', zorder=2)
    
plt.ylim([2.3, 3.5]) # 1 - 3.5
plt.xlim([3, 14]) # 4 - 14
plt.xlabel('I(X;M)')
plt.ylabel('I(Y;M)')
plt.title('Tanh')
    
# colorbar
  
cbaxes = fig.add_axes([1.0, 0.125, 0.03, 0.8]) 
plt.colorbar(sm, label='Epoch', cax=cbaxes)
#plt.colorbar(sm, label='Epoch')


plt.tight_layout()

plt.show()

/var/folders/s8/kjdbnzj17y733ddxj8c_0c080000gn/T/ipykernel_16551/1709962187.py:66: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [311]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.cm as cm

# Mock data for demonstration
n_epoch = IXM_N.shape[0]
epochs = np.load("mi_epoch_4_246.npy")
print(epochs.shape)

COLORBAR_MAX_EPOCHS = 5000

# Set up the figure and color map
fig, ax = plt.subplots(figsize=(8, 8))
sm = plt.cm.ScalarMappable(cmap='gnuplot', norm=mcolors.LogNorm(vmin=1, vmax=COLORBAR_MAX_EPOCHS))

# Plot the data
for epoch in range(1,n_epoch):
    c = sm.to_rgba(epochs[epoch])
    xmvals = IXM_N[epoch]
    ymvals = IYM_N[epoch]
    ax.plot(xmvals, ymvals, c=c, alpha=0.1, zorder=1)
    ax.scatter(xmvals, ymvals, s=40, facecolors=[c], edgecolor='none', zorder=2)

# Set axis limits and labels
ax.set_ylim([1, 3.5])
ax.set_xlim([0, 14])
ax.set_xlabel('I(X;M)')
ax.set_ylabel('I(Y;M)')
ax.set_title('ReLu')

# Add the colorbar
cbaxes = fig.add_axes([0.85, 0.125, 0.03, 0.75])  # Adjust position and size
plt.colorbar(sm, label='Epoch', cax=cbaxes)

# Adjust layout
plt.tight_layout(rect=[0, 0, 0.85, 1])  # Leave space for the colorbar
plt.show()


(109,)


/var/folders/s8/kjdbnzj17y733ddxj8c_0c080000gn/T/ipykernel_16551/2391542994.py:38: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.85, 1])  # Leave space for the colorbar


In [331]:
## plot cluster distances
distances = np.load(f"cluster_distances_4_{seed}.npy")
epochs = np.load(f"mi_epoch_4_{seed}.npy")
# distances = np.load("results/final_tanh_246_sd01/cluster_distances_4_246.npy")
# epochs = np.load("results/final_tanh_246_sd01/mi_epoch_4_246.npy")
print(distances.shape, epochs.shape)

(109, 4, 10) (109,)


In [339]:
fig, ax = plt.subplots(2, 2)
labels = [str(i) for i in range(10)]
ax = ax.ravel()

for i in range(4):
    for j in range(10):
        ax[i].plot(epochs, distances[:,i,j], label=labels[j])
    ax[i].set_title(f'Layer {i+1}')
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Mean Distance')
    ax[i].grid(True)

handles, _ = ax[0].get_legend_handles_labels()
ax[1].set_ylim([0,300])
ax[2].set_ylim([0,300])
ax[3].set_ylim([0,225])

fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.0), fontsize='x-small', ncol=5)
fig.tight_layout(rect=[0, 0.1, 1, 1])  # Reserve space on the right for the legend
plt.show()

In [333]:
fig, ax = plt.subplots(2, 2)
labels = [str(i) for i in range(10)]
ax = ax.ravel()

for i in range(4):
    for j in range(10):
        ax[i].plot(epochs[:41], distances[:41,i,j], label=labels[j])
    ax[i].set_title(f'Layer {i+1}')
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Mean Distance')
    ax[i].grid(True)

handles, _ = ax[0].get_legend_handles_labels()


fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.0), fontsize='x-small', ncol=5)
fig.tight_layout(rect=[0, 0.1, 1, 1])  # Reserve space on the right for the legend
plt.show()

In [313]:
## BINNED ENTROPY QUANTIZED
quant_ent = np.load("binned_entropy_4_246.npy")
print(quant_ent.shape)

(109, 4, 3)


In [316]:
fig, ax = plt.subplots(2, 2)
labels = ["10", "30", "100"]
ax = ax.ravel()

for i in range(4):
    for j in range(3):
        ax[i].plot(epochs, quant_ent[:,i,j], label=labels[j])
    
    ax[i].set_title(f'Layer {i+1}')
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('Quantized Entropy, nats')
    ax[i].grid(True)

ax[0].set_ylim(9,9.3)
handles, _ = ax[0].get_legend_handles_labels()


fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.0), fontsize='x-small', ncol=5)
fig.tight_layout(rect=[0, 0.1, 1, 1])  # Reserve space on the right for the legend
plt.show()

In [317]:
## PLOT TRAINING
train_data = np.load(f"train_4_{seed}.npy")
#train_data = np.load(f"results/4_12/train_4_12.npy")
print(train_data.shape)
train_data[:,[1,4]]

(65, 5)


array([[6.94266667e-01, 0.00000000e+00],
       [9.01850000e-01, 1.00000000e+01],
       [9.40133333e-01, 2.00000000e+01],
       [9.45750000e-01, 3.00000000e+01],
       [9.50800000e-01, 4.00000000e+01],
       [9.52633333e-01, 5.00000000e+01],
       [9.57700000e-01, 6.00000000e+01],
       [9.56850000e-01, 7.00000000e+01],
       [9.60300000e-01, 8.00000000e+01],
       [9.58650000e-01, 9.00000000e+01],
       [9.63316667e-01, 1.00000000e+02],
       [9.62350000e-01, 1.10000000e+02],
       [9.59233333e-01, 1.20000000e+02],
       [9.65333333e-01, 1.30000000e+02],
       [9.65283333e-01, 1.40000000e+02],
       [9.64016667e-01, 1.50000000e+02],
       [9.67566667e-01, 2.00000000e+02],
       [9.72350000e-01, 3.00000000e+02],
       [9.74350000e-01, 4.00000000e+02],
       [9.76716667e-01, 5.00000000e+02],
       [9.78200000e-01, 6.00000000e+02],
       [9.79533333e-01, 7.00000000e+02],
       [9.77433333e-01, 8.00000000e+02],
       [9.80966667e-01, 9.00000000e+02],
       [9.801500

In [318]:
fig, ax = plt.subplots(2)
ax[0].plot(train_data[:,4],train_data[:,0], label="Train loss")
ax[1].plot(train_data[:,4],train_data[:,1], label="Train Acc")
ax[0].plot(train_data[:,4],train_data[:,2], label="Test loss")
ax[1].plot(train_data[:,4],train_data[:,3], label="Test Acc")
ax[0].set_xlabel("Epoch")
ax[1].set_xlabel("Epoch")
ax[0].set_title("Loss")
ax[1].set_title("Accuracy")
ax[0].legend()
ax[1].legend()
fig.tight_layout()
plt.show()

In [319]:
## Plot MI w/ EDGE over epochs
fig, ax = plt.subplots(2)
labels = ["Layer 1", "Layer 2", "Layer 3", "Layer 4"]

for i in range(2):
    for j in range(4):
        ax[i].plot(epochs, a[i,:,j], label=labels[j])
    
    ax[i].set_xlabel('Epoch')
    ax[i].set_ylabel('MI')
    ax[i].grid(True)

ax[0].set_title("MI(X,M), EDGE")
ax[1].set_title("MI(M,Y), EDGE")
handles, _ = ax[0].get_legend_handles_labels()


fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(0.5, -0.0), fontsize='x-small', ncol=5)
fig.tight_layout(rect=[0, 0.1, 1, 1])  # Reserve space on the right for the legend
plt.show()

In [320]:
fig, ax = plt.subplots()
labels = ["Layer 2", "Layer 3", "Layer 4"]
for i in range(3):
    ax.plot(epochs[15:100],a[0,15:100,i+1], label=labels[i])

ax.set_xlabel("Epoch")
ax.set_ylabel("MI")
ax.set_title("MI(X;M), EDGE")
handles, _ = ax.get_legend_handles_labels()
fig.legend()
fig.tight_layout(rect=[0,0,1,1])
plt.show()

In [274]:
from scipy.stats import spearmanr

corr = np.zeros((quant_ent.shape[1],2))
print(corr.shape)
print(quant_ent.shape)
for i in range(quant_ent.shape[1]):
    res = spearmanr(quant_ent[:,i,2],a[0,:,i]) # we use max number of bins since entropy estimate is more reliable
    print(res)
    corr[i,0], corr[i,1] = res.statistic, res.pvalue

print(corr)

(4, 2)
(109, 4, 3)
SignificanceResult(statistic=nan, pvalue=nan)
SignificanceResult(statistic=0.9971550366045779, pvalue=5.656929819723255e-122)
SignificanceResult(statistic=0.9949031600407748, pvalue=1.88292216145391e-108)
SignificanceResult(statistic=0.9695857659160412, pvalue=3.112175061677024e-67)
[[            nan             nan]
 [9.97155037e-001 5.65692982e-122]
 [9.94903160e-001 1.88292216e-108]
 [9.69585766e-001 3.11217506e-067]]


/var/folders/s8/kjdbnzj17y733ddxj8c_0c080000gn/T/ipykernel_16551/1096140510.py:7: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  res = spearmanr(quant_ent[:,i,2],a[0,:,i]) # we use max number of bins since entropy estimate is more reliable
